In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from sklearn.ensemble import RandomForestClassifier

from hyperopt import hp
from hyperopt import fmin, tpe, space_eval
from hyperopt.pyll.base import scope


import mag
from mag.experiment import Experiment
from mag import summarize


from speech2phone.preprocessing.TIMIT.phones import get_data, get_phones
from speech2phone.preprocessing.filters import mel

np.set_printoptions(precision=3)

# Random Forest with `mag`

In [2]:
X_toy, y_toy = get_data(dataset='toy', preprocessor=mel, TIMIT_root='../TIMIT/TIMIT')

Loading toy/mel/0 set from cache... done.


In [3]:
def random_forest_acc(n_estimators, max_depth):
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf.fit(X_toy, y_toy)
    acc = rf.score(X_toy, y_toy)
    return acc



experiments_dir = '../experiments/results/random_forest_v1'
config = {
    'n_estimators': 11,
    'max_depth': 10
}
with Experiment(config=config, experiments_dir=experiments_dir) as experiment:
    config = experiment.config
    acc = random_forest_acc(config.n_estimators, config.max_depth)
    
    experiment.register_result('accuracy', acc)

# Random Forest with `hyperopt`

In [4]:
def acc_wrapper(args):
    acc = random_forest_acc(args['n_estimators'], args['max_depth'])
    print("{}: {:.3f}".format(args, acc))
    return -acc

space = {
    'n_estimators': scope.int(hp.uniform('n_estimators', 1, 100)),
    'max_depth': scope.int(hp.uniform('max_depth', 1, 30))

}
best = fmin(acc_wrapper, space, algo=tpe.suggest, max_evals=50)
print("")
print("Best Raw:", best)
print("Best Readable:", space_eval(space, best))

{'max_depth': 21, 'n_estimators': 89}: 1.000
{'max_depth': 13, 'n_estimators': 77}: 0.987
{'max_depth': 11, 'n_estimators': 75}: 0.905
{'max_depth': 28, 'n_estimators': 63}: 1.000
{'max_depth': 24, 'n_estimators': 32}: 1.000
{'max_depth': 7, 'n_estimators': 16}: 0.432
{'max_depth': 11, 'n_estimators': 86}: 0.906
{'max_depth': 28, 'n_estimators': 51}: 1.000
{'max_depth': 28, 'n_estimators': 86}: 1.000
{'max_depth': 7, 'n_estimators': 10}: 0.419
{'max_depth': 17, 'n_estimators': 36}: 1.000
{'max_depth': 17, 'n_estimators': 63}: 1.000
{'max_depth': 6, 'n_estimators': 59}: 0.378
{'max_depth': 21, 'n_estimators': 89}: 1.000
{'max_depth': 28, 'n_estimators': 61}: 1.000
{'max_depth': 1, 'n_estimators': 21}: 0.128
{'max_depth': 26, 'n_estimators': 32}: 1.000
{'max_depth': 17, 'n_estimators': 4}: 0.885
{'max_depth': 1, 'n_estimators': 56}: 0.128
{'max_depth': 12, 'n_estimators': 55}: 0.964
{'max_depth': 22, 'n_estimators': 99}: 1.000
{'max_depth': 24, 'n_estimators': 40}: 1.000
{'max_depth': 20

# Random Forest with `mag` and `hyperopt`

In [5]:
def acc_wrapper(args):
    experiments_dir = '../experiments/results/random_forest_v2'
    with Experiment(config=args, experiments_dir=experiments_dir) as experiment:
        config = experiment.config
        acc = random_forest_acc(config.n_estimators, config.max_depth)
        experiment.register_result('accuracy', acc)
    return -acc

space = {
    'n_estimators': scope.int(hp.uniform('n_estimators', 1, 100)),
    'max_depth': scope.int(hp.uniform('max_depth', 1, 30))

}
best = fmin(acc_wrapper, space, algo=tpe.suggest, max_evals=5)
print("")
print("Best Raw:", best)
print("Best Readable:", space_eval(space, best))


Best Raw: {'max_depth': 22.230556326650923, 'n_estimators': 64.26917198881927}
Best Readable: {'max_depth': 22, 'n_estimators': 64}
